In [20]:
import re
import pandas as pd


In [23]:
def process_filings(dataframe):

    columns = columns_10K
    header_mappings = header_mappings_10K

    df = dataframe
    items = pd.DataFrame(columns = columns, dtype=object)
    for i in df.index:
        form_text = df.text[i]
        item_iter = get_form_items(form_text)
        items.loc[i] = items_to_df_row(item_iter, columns)
    items.rename(columns=header_mappings, inplace=True)
    df = pd.merge(df, items, left_index=True, right_index=True)
    return df

In [2]:
pip install edgartools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 3.1 MB/s eta 0:00:00


In [6]:
import pandas as pd
from edgar import Company

# Define the company ticker
ticker = "AAPL"

# Define the range of years for which you want to retrieve 10-K filings
start_year = 1995
end_year = 2023

# Initialize an empty DataFrame to store all items
all_items_df = pd.DataFrame(columns=["Item", "Content"])

# Retrieve 10-K filings for the specified range of years
filings = Company(ticker).get_filings(form="10-K")
filings.filter(date=f"{start_year}-01-01:{end_year}-12-31")

# Iterate over each filing for the current year
for filing in filings:
    # Retrieve the HTML content of the filing
    html = filing.html()

    if html:
        # Create a ChunkedDocument object from the HTML content
        chunked_document = ChunkedDocument(html)

        # Extract items (sections) from the filing based on a condition
        items = chunked_document.show_items("Item.str.contains('ITEM', case=False)", "Item")

        # Initialize a dictionary to store content for each item
        item_contents = {}

        # Iterate over each item and extract content
        for index, row in items.iterrows():
            # Store the content in reverse order to correct the concatenation
            item_contents[row["Item"]] = item_contents.get(row["Item"], "") + row["Text"]

        # Create a DataFrame from the item contents
        filing_df = pd.DataFrame.from_dict(item_contents, orient="index", columns=["Content"])
        filing_df.reset_index(inplace=True)
        filing_df.columns = ["Item", "Content"]

        # Concatenate the extracted items to the DataFrame containing all items
        all_items_df = pd.concat([all_items_df, filing_df], ignore_index=True)

# Print the DataFrame containing all items
# Define the list of valid items
valid_items = ["Item 1", "Item 1A", "Item 7"]

# Filter the DataFrame to keep only the rows with valid items
filtered_df = all_items_df[all_items_df['Item'].isin(valid_items)]

# Print the filtered DataFrame
print(filtered_df)

        Item                                            Content
0     Item 1  Item 1.    Business\nCompany Background\nThe C...
1    Item 1A  Item 1A.    Risk Factors\nThe Company’s busine...
9     Item 7  Item 7.    Management’s Discussion and Analysi...
23    Item 1  Item 1.    Business\nCompany Background\nThe C...
24   Item 1A  Item 1A.    Risk Factors\nThe Company’s busine...
31    Item 7  Item 7.    Management’s Discussion and Analysi...
45    Item 1  Item 1.    Business\nCompany Background\nThe C...
46   Item 1A  Item 1A.    Risk Factors\nThe Company’s busine...
53    Item 7  Item 7.    Management’s Discussion and Analysi...
67    Item 1  Item 1.    Business\nCompany Background\nThe C...
68   Item 1A  Item 1A.    Risk Factors\nThe following discus...
75    Item 7  Item 7.    Management’s Discussion and Analysi...
88    Item 1  Item 1. | BusinessCompany Background\nThe Comp...
89   Item 1A  Item 1A. | Risk FactorsThe following discussio...
96    Item 7  Item 7. | Management’s Dis

In [243]:
import pandas as pd
from rich import print

from edgar import Filing
from edgar import find
from edgar.company_reports import TenK, TenQ, TwentyF, EightK
from edgar.htmltools import ChunkedDocument

# Find and retrieve the filing
filing = find("0000320193-23-000106")
html = filing.html()

# Create a ChunkedDocument object from the HTML content
chunked_document = ChunkedDocument(html)

# Extract items (sections) from the filing based on a condition
items = chunked_document.show_items("Item.str.contains('ITEM', case=False)", "Item")

# Initialize a dictionary to store content for each item
item_contents = {}

# Iterate over each item and extract content
for index, row in items.iterrows():
    # Store the content in reverse order to correct the concatenation
    item_contents[row["Item"]] = item_contents.get(row["Item"], "")+ row["Text"]

# Create a DataFrame from the item contents
combined_df = pd.DataFrame.from_dict(item_contents, orient="index", columns=["Content"])

# Reset index to make items a column
combined_df.reset_index(inplace=True)
combined_df.columns = ["Item", "Content"]

# Print the DataFrame
print(combined_df)

Item                                            Content
0    Item 1  Item 1.    Business\nCompany Background\nThe C...
1   Item 1A  Item 1A.    Risk Factors\nThe Company’s busine...
2   Item 1B       Item 1B.    Unresolved Staff Comments\nNone.
3   Item 1C  Item 1C.    CybersecurityNot applicable.Apple ...
4    Item 2  Item 2.    Properties\nThe Company’s headquart...
5    Item 3  Item 3.    Legal Proceedings\nEpic Games\nEpic...
6    Item 4  Item 4.    Mine Safety DisclosuresNot applicab...
7    Item 5  Item 5.    Market for Registrant’s Common Equi...
8    Item 6  Item 6.    [Reserved]\nApple Inc. | 2023 Form ...
9    Item 7  Item 7.    Management’s Discussion and Analysi...
10  Item 7A  Item 7A.    Quantitative and Qualitative Discl...
11   Item 8  Item 8.    Financial Statements and Supplement...
12   Item 9  Item 9.    Changes in and Disagreements with A...
13  Item 9A  Item 9A.    Controls and Procedures\nEvaluatio...
14  Item 9B  Item 9B.    Other Information\nInsider Trading...
15  Item 9C  Item 9C.    Disclosure Regarding Foreign Juris...
16  Item 10  Item 10.    Directors, Executive Officers and ...
17  Item 11  Item 11.    Executive CompensationThe informat...
18  Item 12  Item 12.    Security Ownership of Certain Bene...
19  Item 13  Item 13.    Certain Relationships and Related ...
20  Item 14  Item 14.    Principal Accountant Fees and Serv...
21  Item 15  Item 15.    Exhibit and Financial Statement Sc...
22  Item 16  Item 16.    Form 10-K Summary\nNone.Apple Inc....

In [7]:
%pip -q install -q pandas nltk transformers

In [8]:
import nltk
import pandas as pd
from transformers import pipeline

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
summarizer = pipeline('summarization', model='t5-base')

classifier_model_name = 'bhadresh-savani/distilbert-base-uncased-emotion'
classifier_emotions = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

classifier = pipeline('text-classification', model=classifier_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [11]:
def find_emotional_sentences(text, emotions, threshold):
    sentences_by_emotion = {}
    for e in emotions:
        sentences_by_emotion[e]=[]
    sentences = nltk.sent_tokenize(text)
    print(f'Document has {len(text)} characters and {len(sentences)} sentences.')
    for s in sentences:
        prediction = classifier(s)
        if (prediction[0]['label']!='neutral' and prediction[0]['score']>threshold):
            #print (f'Sentence #{sentences.index(s)}: {prediction} {s}')
            sentences_by_emotion[prediction[0]['label']].append(s)
    for e in emotions:
        print(f'{e}: {len(sentences_by_emotion[e])} sentences')
    return sentences_by_emotion

In [16]:
def summarize_sentences(sentences_by_emotion, min_length, max_length):
    all_summaries = []
    for k in sentences_by_emotion.keys():
        if len(sentences_by_emotion[k]) != 0:
            text = ' '.join(sentences_by_emotion[k])
            summary = summarizer(text, min_length=min_length, max_length=max_length)
            all_summaries.append(summary[0]['summary_text'])  # Append the summary to the list
    return all_summaries

In [13]:
#1 Row Example
text = filtered_df.iloc[0]['Content']

sentences_by_emotion = find_emotional_sentences(text, classifier_emotions, 0.7)
summarize_sentences(sentences_by_emotion, min_length=20, max_length=50)

Document has 15090 characters and 79 sentences.
anger: 4 sentences
disgust: 0 sentences
fear: 0 sentences
joy: 65 sentences
sadness: 2 sentences
surprise: 0 sentences
ANGER: competition has been particularly intense as competitors have aggressively cut prices and lowered product margins . many of the Company’s competitors seek to compete primarily through aggressive pricing and very low cost structures .

JOY: the company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessories, and sells a variety of related services . the company’s fiscal year is the 52- or 53-week period that ends

SADNESS: many components used by the Company, including those that are available from multiple sources, are at times subject to industry-wide shortage and significant commodity pricing fluctuations . further, the Company’s references to website URLs are intended to be inactive



In [17]:
# Initialize dictionaries to store aggregated emotional sentences and their summaries
all_sentences_by_emotion = {emotion: [] for emotion in classifier_emotions}
all_summaries = []

# Iterate over each row in the filtered DataFrame
for index, row in filtered_df.iterrows():
    # Get the content from the current row
    text = row['Content']

    # Find emotional sentences in the text
    sentences_by_emotion = find_emotional_sentences(text, classifier_emotions, 0.7)

    # Summarize emotional sentences
    summaries = summarize_sentences(sentences_by_emotion, min_length=20, max_length=50)

    # Aggregate emotional sentences and summaries
    for emotion, sentences in sentences_by_emotion.items():
        all_sentences_by_emotion[emotion].extend(sentences)

    # Aggregate summaries
    all_summaries.extend(summaries)

# Generate final insight based on aggregated data
final_insight = {
    "emotional_sentences": all_sentences_by_emotion,
    "summaries": all_summaries
}

# Print or return the final insight
print(final_insight)

Document has 15090 characters and 79 sentences.
anger: 4 sentences
disgust: 0 sentences
fear: 0 sentences
joy: 65 sentences
sadness: 2 sentences
surprise: 0 sentences
Document has 67874 characters and 292 sentences.
anger: 19 sentences
disgust: 0 sentences
fear: 23 sentences
joy: 103 sentences
sadness: 19 sentences
surprise: 0 sentences
Document has 16187 characters and 63 sentences.
anger: 0 sentences
disgust: 0 sentences
fear: 4 sentences
joy: 34 sentences
sadness: 5 sentences
surprise: 0 sentences
Document has 14638 characters and 72 sentences.
anger: 4 sentences
disgust: 0 sentences
fear: 1 sentences
joy: 58 sentences
sadness: 2 sentences
surprise: 0 sentences


Your max_length is set to 50, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


Document has 70786 characters and 318 sentences.
anger: 20 sentences
disgust: 0 sentences
fear: 24 sentences
joy: 109 sentences
sadness: 20 sentences
surprise: 0 sentences
Document has 18341 characters and 67 sentences.


Token indices sequence length is longer than the specified maximum sequence length for this model (693 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (693) must match the size of tensor b (512) at non-singleton dimension 1

In [251]:
pip install -U sec-edgar-downloader

In [255]:
from sec_edgar_downloader import Downloader
import re
import os

def extract_financial_data(text):
    # Define regular expressions to match net income and EPS
    net_income_regex = r"CONSOLIDATED\s+STATEMENTS\s+OF\s+OPERATIONS.*?Net\s+income.*?(\$\s*\d[\d,.\s]*\d)"
    eps_regex = r"CONSOLIDATED\s+STATEMENTS\s+OF\s+OPERATIONS.*?Earnings\s+per\s+share.*?(\$\s*\d[\d,.\s]*\d)"

    # Search for net income and EPS in the text
    net_income_matches = re.findall(net_income_regex, text, re.DOTALL)
    eps_matches = re.findall(eps_regex, text, re.DOTALL)

    # Process net income matches
    net_income = None
    for income_match in net_income_matches:
        net_income_str = income_match.strip()  # Remove leading/trailing spaces
        # Extract numeric value from the net income string
        net_income = float(re.sub(r'[^\d.]', '', net_income_str))
        if net_income != 1e-05 and net_income != 205.0 and net_income>1000:
            break  # Stop searching after finding a valid net income value

    # Process EPS matches
    eps = None
    for eps_match in eps_matches:
        eps_str = eps_match.strip()  # Remove leading/trailing spaces
        # Extract numeric value from the EPS string
        eps = float(re.sub(r'[^\d.]', '', eps_str))
        if eps != 1e-05 and eps != 205.0 and eps > 0 and eps <15:
            break  # Stop searching after finding a valid EPS value

    return net_income, eps

def download_10k_filings(company_ticker):
    downloader = Downloader("MaxTest", "mli751@gatech.edu")
    downloaded_filings = []
    years_with_no_filings = []
    net_income_list = []
    eps_list = []



    filings = downloader.get("10-K", company_ticker, after=f"{1995}-01-01", before=f"{2024}-01-01")
    # Download filings for each year in the specified range
            # Attempt to download filings for the given year

    if filings:
        # If filings were found, extract financial data from each filing
        directory = f"sec-edgar-filings/{company_ticker.upper()}/10-K/"

    # Loop over files in the directory
        for root, dirs, files in os.walk(directory):
            for filename in files:
                if filename == "full-submission.txt":
                    filepath = os.path.join(root, filename)
                    # Read the contents of the file
                    with open(filepath, "r") as file:
                        text = file.read()
                        # Process the content as needed
                        net_income, eps = extract_financial_data(text)
                        if net_income is not None and eps is not None:
                            net_income_list.append(net_income)
                            eps_list.append(eps)
                            print(f"Extracted financial data from {company_ticker} filing in {filepath}: Net Income = {net_income}, EPS = {eps}")
                        else:
                            print(f"No financial data found in {company_ticker} filing in{filepath}")

    return net_income_list, eps_list

if __name__ == "__main__":
    company_tickers = ["AAPL"]  # Add your desired tickers here
    final_net_income_list = []
    final_eps_list = []
    for ticker in company_tickers:
        print(f"Downloading 10-K filings for {ticker}...")
        net_income_list, eps_list = download_10k_filings(ticker)
        final_net_income_list.extend(net_income_list)
        final_eps_list.extend(eps_list)

    print("Final Net Income List:", final_net_income_list)
    print("Final EPS List:", final_eps_list)

Downloading 10-K filings for AAPL...

Extracted financial data from AAPL filing in sec-edgar-filings/AAPL/10-K/0000320193-20-000096/full-submission.txt: 
Net Income = 57411.0, EPS = 3.31

No financial data found in AAPL filing insec-edgar-filings/AAPL/10-K/0000320193-95-000016/full-submission.txt

Extracted financial data from AAPL filing in sec-edgar-filings/AAPL/10-K/0001193125-13-416534/full-submission.txt: 
Net Income = 35.0, EPS = 35.0

Extracted financial data from AAPL filing in sec-edgar-filings/AAPL/10-K/0000320193-22-000108/full-submission.txt: 
Net Income = 99803.0, EPS = 6.15

...

No financial data found in AAPL filing insec-edgar-filings/AAPL/10-K/0000320193-96-000023/full-submission.txt

No financial data found in AAPL filing insec-edgar-filings/AAPL/10-K/0001047469-02-007674/full-submission.txt

Extracted financial data from AAPL filing in sec-edgar-filings/AAPL/10-K/0000320193-23-000106/full-submission.txt: 
Net Income = 96995.0, EPS = 6.16

Final Net Income List:
[57411.0, 35.0, 99803.0, 53394.0, 13686.0, 48351.0, 41733.0, 45687.0, 94680.0, 59531.0, 39510.0, 96995.0]

Final EPS List:
[3.31, 35.0, 6.15, 9.28, 3.05, 9.27, 44.64, 8.35, 5.67, 12.01, 6.49, 6.16]